In [57]:
import pandas as pd
#from pandas_profiling import ProfileReport
from persiantools.jdatetime import JalaliDate
from persiantools.jdatetime import JalaliDateTime
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import seaborn as sns
import datetime, pytz
import math
import numpy as np

In [2]:
Data=pd.read_excel("/media/sda6/Projects/Data Mining on gas Network/output.xls",dtype={'TOF':"category", 'AF':"category",
       'DS':"category", 'feeder':"category",'TOP':"category", 'Relay':"category", 'COB':"category", 'PD':"category",
       'COP':"category","DOFDB":"float64"})

In [3]:
time=list()
for i in Data['DFB']:
    i=i.split()
    yeardaymonth=i[0].split("/")
    hourandminute=i[1].split(":")
    time.append(JalaliDateTime(int(yeardaymonth[0]),int(yeardaymonth[1]),int(yeardaymonth[2]),int(hourandminute[0]),int(hourandminute[1])).to_gregorian())
Data['DFB']=time

In [4]:
time=list()
for i in Data['CFB']:
    try:
        i=i.split()
        yeardaymonth=i[0].split("/")
        hourandminute=i[1].split(":")
        time.append(JalaliDateTime(int(yeardaymonth[0]),int(yeardaymonth[1]),int(yeardaymonth[2]),int(hourandminute[0]),int(hourandminute[1])).to_gregorian())
    except:
        time.append(float("NaN"))
Data['CFB']=time

In [5]:
time=list()
for i in Data['PS']:
    try:
        i=i.split()
        yeardaymonth=i[0].split("/")
        hourandminute=i[1].split(":")
        time.append(JalaliDateTime(int(yeardaymonth[0]),int(yeardaymonth[1]),int(yeardaymonth[2]),int(hourandminute[0]),int(hourandminute[1])).to_gregorian())
    except:
        time.append(float("NaN"))
Data['PS']=time

In [6]:
print(Data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1649 entries, 0 to 1648
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   BN      1649 non-null   int64         
 1   TOF     1649 non-null   category      
 2   AF      1649 non-null   category      
 3   DS      1649 non-null   category      
 4   feeder  1649 non-null   category      
 5   RC      1649 non-null   float64       
 6   TOP     1649 non-null   category      
 7   Relay   1649 non-null   category      
 8   COB     1649 non-null   category      
 9   PD      858 non-null    category      
 10  COP     807 non-null    category      
 11  DFB     1649 non-null   datetime64[ns]
 12  CFB     1646 non-null   datetime64[ns]
 13  DOFDB   1649 non-null   float64       
 14  PS      812 non-null    datetime64[ns]
 15  STOLP   813 non-null    float64       
 16  EENS    1649 non-null   float64       
dtypes: category(9), datetime64[ns](3), float64(4), int64

In [46]:
Data=Data.sort_values("DFB",ignore_index=True)

In [50]:
Data

,BN,TOF,AF,DS,feeder,RC,TOP,Relay,COB,PD,COP,DFB,CFB,DOFDB,PS,STOLP,EENS
0,88644,A,A,A,A,0.177419,A,A,A,NaN,NaN,2019-03-21 06:54:00,2019-03-21 06:59:00,5.0,NaT,NaN,0.010946
1,88645,A,B,B,B,0.145161,A,A,B,B,B,2019-03-21 07:15:00,2019-03-21 07:17:00,2.0,NaT,NaN,0.003127
2,88648,A,A,C,C,0.032258,A,A,C,NaN,NaN,2019-03-21 12:03:00,2019-03-21 12:45:00,42.0,NaT,NaN,0.016419
3,88657,A,A,A,D,0.048387,A,A,D,C,C,2019-03-22 01:20:00,2019-03-22 02:25:00,65.0,2019-03-22 08:15:00,415.0,0.039093
4,88664,A,C,D,A,0.161290,A,B,E,D,D,2019-03-22 16:26:00,2019-03-22 16:50:00,24.0,2019-03-22 22:00:00,334.0,0.075059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1644,96326,A,L,BB,G,0.096774,A,A,F,C,C,2020-03-19 10:21:00,2020-03-19 10:30:00,9.0,2020-03-19 10:40:00,19.0,0.017201
1645,96335,A,C,E,C,0.193548,A,A,I,NaN,NaN,2020-03-19 13:23:00,2020-03-19 14:15:00,52.0,2020-03-19 13:34:00,11.0,0.025020
1646,96346,A,H,M,E,0.274194,A,A,R,NaN,NaN,2020-03-19 18:28:00,2020-03-19 19:07:00,39.0,2020-03-19 19:52:00,84.0,0.193120
1647,96347,A,H,X,C,0.258065,A,A,X,NaN,NaN,2020-03-19 19:08:00,2020-03-19 19:10:00,2.0,NaT,NaN,0.006255


In [51]:
def diff(FEEDER,DATA):
    DATA=DATA[DATA.feeder==FEEDER].reset_index().sort_values("DFB",ignore_index=True)
    pois=list()
    for i in range(DATA.shape[0]-1):
        pois.append((DATA.DFB[i+1]-DATA.DFB[i]).days)
    return pois

In [63]:
for i in pd.unique(Data.feeder):
    print(i,": ",np.mean(diff(i,Data)))

A :  1.4919786096256684
B :  6.037735849056604
C :  1.6534090909090908
D :  1.4947368421052631
E :  3.2448979591836733
F :  2.1956521739130435
G :  2.393700787401575
H :  2.4796747967479673
I :  1.4375
J :  2.6206896551724137
K :  1.9337748344370862
L :  6.113207547169812
M :  18.470588235294116
N :  40.57142857142857
O :  47.166666666666664
